## An end-to-end pipeline for the analysis of HCP diffusion MRI data using Dipy

From the S3 bucket to tract profiles

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os.path as op
%matplotlib inline

### Get the data from AWS S3

We assume that you have a file '.aws/credentials', that includes a section: 

    [hcp]
    AWS_ACCESS_KEY_ID=XXXXXXXXXXXXXXXX
    AWS_SECRET_ACCESS_KEY=XXXXXXXXXXXXXXXX
with the credentials you got [from HCP](https://wiki.humanconnectome.org/display/PublicData/How+To+Connect+to+Connectome+Data+via+AWS)

In [2]:
data_files = {'./bvals':'HCP/994273/T1w/Diffusion/bvals', 
              './bvecs':'HCP/994273/T1w/Diffusion/bvecs', 
              './data.nii.gz':'HCP/994273/T1w/Diffusion/data.nii.gz'}

In [3]:
import botocore.session
import boto3
boto3.setup_default_session(profile_name='hcp')
s3 = boto3.resource('s3')
bucket = s3.Bucket('hcp-openaccess')

In [4]:
for k in data_files.keys():
    if not op.exists(k):
        bucket.download_file(data_files[k], k)

In [5]:
import dipy.core.gradients as dpg
import nibabel as nib

In [6]:
# Read in the gradient table, setting the threshold to 10, so that low values are considered as 0:
gtab = dpg.gradient_table('./bvals', './bvecs', b0_threshold=10)

In [7]:
img = nib.load('./data.nii.gz')

In [8]:
data = img.get_data()

### Brain extraction (median Otsu)

See (example and explanations)

In [9]:
# We look at the average non diffusion-weighted as data for brain extraction
mean_b0 = np.mean(data[..., gtab.b0s_mask], -1)

In [10]:
from dipy.segment.mask import median_otsu


/home/ubuntu/anaconda3/lib/python3.5/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


In [11]:
if not op.exists('./brain_mask.nii.gz'):
    from dipy.segment.mask import median_otsu
    _, brain_mask = median_otsu(mean_b0, 4, 2, False, vol_idx=np.where(gtab.b0s_mask), dilate=1)
    nib.save(nib.Nifti1Image(brain_mask.astype(int), img.affine), 'brain_mask.nii.gz')
else: 
    brain_mask = nib.load('brain_mask.nii.gz').get_data()

### Denoising (NLMEANS)

See [example and explanations](http://nipy.org/dipy/examples_built/denoise_nlmeans.html#example-denoise-nlmeans)


In [12]:
if not op.exists('./denoised_data.nii.gz'):
    from dipy.denoise import nlmeans
    from dipy.denoise.noise_estimate import estimate_sigma
    sigma = estimate_sigma(data)
    denoised_data = nlmeans.nlmeans(data, sigma=sigma, mask=brain_mask)
    nib.save(nib.Nifti1Image(denoised_data, img.affine), 'denoised_data.nii.gz')
else:
    denoised_data = nib.load('denoised_data.nii.gz').get_data()

### Modeling (DTI)

See [example and explanations](http://nipy.org/dipy/examples_built/reconst_dti.html#example-reconst-dti)

In [13]:
if not op.exists('./dti_fa.nii.gz'):
    import dipy.reconst.dti as dti
    ten_model = dti.TensorModel(gtab)
    ten_fit = ten_model.fit(denoised_data, mask=brain_mask)
    fa = ten_fit.fa 
    md = ten_fit.md
    nib.save(nib.Nifti1Image(fa, img.affine), 'dti_fa.nii.gz')
    nib.save(nib.Nifti1Image(md, img.affine), 'dti_md.nii.gz')
else:
    fa = nib.load('./dti_fa.nii.gz').get_data()
    md = nib.load('./dti_md.nii.gz').get_data()

### Segmentation (FA)

FA is a measure of the anisotropy of diffusion in each voxel in the image. Because white matter tend to have higher anisotropy, we can use it as a way to roughly segment the image into portions that contain white matter and the non-white matter portion of the image. This allows us to reduce the computational demand of subsequent steps by only performing them in the masked regions of the image.


In [14]:
wm_mask = np.zeros(fa.shape, dtype=bool)
wm_mask[fa>0.2] = 1

### Modeling (CSA) and tracking


In [20]:
if not op.exists('./csa_tracks.trk'):
    from dipy.reconst.shm import CsaOdfModel
    from dipy.data import default_sphere
    from dipy.direction import peaks_from_model
    from dipy.tracking.local import LocalTracking
    from dipy.io.trackvis import save_trk

    csa_model = CsaOdfModel(gtab, sh_order=6)
    csa_peaks = peaks_from_model(csa_model, denoised_data, default_sphere,
                                 relative_peak_threshold=.8,
                                 min_separation_angle=45,
                                 mask=wm_mask)

    from dipy.tracking.local import ThresholdTissueClassifier

    classifier = ThresholdTissueClassifier(csa_peaks.gfa, .1)

    from dipy.tracking import utils
    seeds = utils.seeds_from_mask(wm_mask, density=[2, 2, 2], affine=img.affine)

    # Initialization of LocalTracking. The computation happens in the next step.
    streamlines = LocalTracking(csa_peaks, classifier, seeds, img.affine, step_size=.5)

    # Compute streamlines and store as a list.
    streamlines = [s for s in streamlines if s.shape[0]>10]

    save_trk("csa_tracks.trk", streamlines, img.affine, fa.shape)
else:
    tv = nib.trackvis.read("csa_tracks.trk")
    streamlines = [t[0] for t in tv[0]]

###  Registration to a template

In [ ]:
from dipy.data import read_mni_template
from dipy.align.metrics import CCMetric
from dipy.align.imwarp import SymmetricDiffeomorphicRegistration

MNI_T2 = read_mni_template()
MNI_T2_data = MNI_T2.get_data()
MNI_T2_affine = MNI_T2.get_affine()
use_metric = CCMetric(3, sigma_diff=2.0)
sdr = SymmetricDiffeomorphicRegistration(use_metric, [10, 10, 5],
                                         step_length=0.25)
mapping = sdr.optimize(MNI_T2_data, mean_b0,
                       static_grid2world=img.affine,
                       moving_grid2world=MNI_T2_affine,
                       prealign=None)

In [25]:
LOCC_ni = nib.load(os.path.join(afqpath,'~/.AFQ/templates/callosum2/L_Occipital.nii.gz'))
ROCC_ni = nib.load(os.path.join(afqpath,'~/.AFQ/templates/callosum2/R_Occipital.nii.gz'))
midsag_ni = nib.load(os.path.join(afqpath,'~/.AFQ/templates/callosum2/Callosum_midsag.nii.gz'))


NameError: name 'os' is not defined